<a href="https://colab.research.google.com/github/ANAS1211/ML_pyspark/blob/main/pipeline_random_forrest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

In [3]:
from pyspark.sql import SparkSession

In [4]:
from pyspark import SparkContext

In [5]:
sc = SparkContext.getOrCreate()

In [6]:
# Construction d'une Session Spark
spark = SparkSession \
    .builder \
    .appName("Pipelines Spark ML") \
    .getOrCreate()

spark

In [7]:
# (a) Importer le fichier HR_comma_sep.csv dans un DataFrame nommé hr.
# (b) Afficher un extrait du DataFrame hr.

In [8]:
import pandas as pd

In [9]:
hr = pd.read_csv("/content/HR_comma_sep.csv")

In [10]:
# Chargement du fichier 'HR_comma_sep.csv'
hr = spark.read.csv('HR_comma_sep.csv', header = True)
# Affichage d'un extrait du DataFrame
hr.sample(False, 0.001, seed=222).toPandas()


,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.87,0.71,3,132,2,0,0,0,sales,low
1,0.88,0.86,3,187,3,0,0,0,marketing,low
2,0.59,0.77,3,169,3,0,0,0,IT,low
3,0.96,0.51,3,237,4,0,0,0,technical,high
4,0.67,0.58,5,161,3,1,0,0,support,low
5,0.87,0.95,3,242,5,0,0,0,sales,low
6,0.9,0.85,4,279,6,0,0,0,support,medium
7,0.92,0.77,5,191,2,0,0,0,technical,medium
8,0.62,0.78,3,178,3,1,0,0,technical,low
9,0.83,0.89,4,136,3,0,0,0,sales,medium


In [11]:
hr.columns
['satisfaction_level',
 'last_evaluation',
 'number_project',
 'average_montly_hours',
 'time_spend_company',
 'Work_accident',
 'left',
 'promotion_last_5years',
 'sales',
 'salary']

['satisfaction_level',
 'last_evaluation',
 'number_project',
 'average_montly_hours',
 'time_spend_company',
 'Work_accident',
 'left',
 'promotion_last_5years',
 'sales',
 'salary']

In [12]:
hr=hr.select('left','satisfaction_level',
 'last_evaluation',
 'number_project',
 'average_montly_hours',
 'time_spend_company',
 'Work_accident',
 'promotion_last_5years',
 'sales',
 'salary')

In [13]:
# Affichage d'une description des variables
hr.describe().toPandas()

,summary,left,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,sales,salary
0,count,14999,14999,14999,14999,14999,14999,14999,14999,14999,14999
1,mean,0.2380825388359224,0.6128335222348166,0.7161017401159978,3.80305353690246,201.0503366891126,3.498233215547703,0.1446096406427095,0.021268084538969265,None,None
2,stddev,0.42592409938029885,0.24863065106114257,0.17116911062327556,1.2325923553183513,49.94309937128406,1.4601362305354808,0.35171855238017957,0.1442814645785825,None,None
3,min,0,0.09,0.36,2,100,10,0,0,IT,high
4,max,1,1,1,7,99,8,1,1,technical,medium


In [14]:
hr.show(2)

+----+------------------+---------------+--------------+--------------------+------------------+-------------+---------------------+-----+------+
|left|satisfaction_level|last_evaluation|number_project|average_montly_hours|time_spend_company|Work_accident|promotion_last_5years|sales|salary|
+----+------------------+---------------+--------------+--------------------+------------------+-------------+---------------------+-----+------+
|   1|              0.38|           0.53|             2|                 157|                 3|            0|                    0|sales|   low|
|   1|               0.8|           0.86|             5|                 262|                 6|            0|                    0|sales|medium|
+----+------------------+---------------+--------------+--------------------+------------------+-------------+---------------------+-----+------+
only showing top 2 rows



In [15]:
# (e) Importer la fonction StringIndexer depuis le package pyspark.ml.feature.
from pyspark.ml.feature import StringIndexer
# (f) Créer un indexeur SalesIndexer transformant une variable sales en une variable indexedSales.
# Première étape: création de l'indexeur.
SalesIndexer = StringIndexer(inputCol='sales', outputCol='indexedSales').fit(hr)

# Deuxième étape: application au DataFrame.
hrSalesIndexed = SalesIndexer.transform(hr)
# (g) Indexer la variable sales de hr dans un nouveau DataFrame nommé hrSalesIndexed.
# (h) Afficher un extrait de hrSalesIndexed.
type(hrSalesIndexed)


pyspark.sql.dataframe.DataFrame

In [16]:
hrSalesIndexed.sample(False, 0.001, seed=222).toPandas()

,left,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,sales,salary,indexedSales
0,0,0.87,0.71,3,132,2,0,0,sales,low,0.0
1,0,0.88,0.86,3,187,3,0,0,marketing,low,5.0
2,0,0.59,0.77,3,169,3,0,0,IT,low,3.0
3,0,0.96,0.51,3,237,4,0,0,technical,high,1.0
4,0,0.67,0.58,5,161,3,1,0,support,low,2.0
5,0,0.87,0.95,3,242,5,0,0,sales,low,0.0
6,0,0.9,0.85,4,279,6,0,0,support,medium,2.0
7,0,0.92,0.77,5,191,2,0,0,technical,medium,1.0
8,0,0.62,0.78,3,178,3,1,0,technical,low,1.0
9,0,0.83,0.89,4,136,3,0,0,sales,medium,0.0


In [17]:
from pyspark.ml.feature import IndexToString

In [18]:
SalaryIndexer= StringIndexer(inputCol='salary', outputCol='indexedSalary').fit(hr)
hrSalaryIndexed= SalaryIndexer.transform(hr)
hrSalaryIndexed.sample(False, 0.001, seed=222).toPandas()

,left,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,sales,salary,indexedSalary
0,0,0.87,0.71,3,132,2,0,0,sales,low,0.0
1,0,0.88,0.86,3,187,3,0,0,marketing,low,0.0
2,0,0.59,0.77,3,169,3,0,0,IT,low,0.0
3,0,0.96,0.51,3,237,4,0,0,technical,high,2.0
4,0,0.67,0.58,5,161,3,1,0,support,low,0.0
5,0,0.87,0.95,3,242,5,0,0,sales,low,0.0
6,0,0.9,0.85,4,279,6,0,0,support,medium,1.0
7,0,0.92,0.77,5,191,2,0,0,technical,medium,1.0
8,0,0.62,0.78,3,178,3,1,0,technical,low,0.0
9,0,0.83,0.89,4,136,3,0,0,sales,medium,1.0


In [19]:
from pyspark.ml import Pipeline

In [20]:
# Création d'une Pipeline
indexer = Pipeline(stages =  [SalaryIndexer, SalesIndexer])
# Indexer les variables de hr
hrIndexed = indexer.fit(hr).transform(hr)
# Affichage d'un extrait de hrIndexed
hrIndexed.sample(False, 0.001 , seed = 222).toPandas()

,left,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,sales,salary,indexedSalary,indexedSales
0,0,0.87,0.71,3,132,2,0,0,sales,low,0.0,0.0
1,0,0.88,0.86,3,187,3,0,0,marketing,low,0.0,5.0
2,0,0.59,0.77,3,169,3,0,0,IT,low,0.0,3.0
3,0,0.96,0.51,3,237,4,0,0,technical,high,2.0,1.0
4,0,0.67,0.58,5,161,3,1,0,support,low,0.0,2.0
5,0,0.87,0.95,3,242,5,0,0,sales,low,0.0,0.0
6,0,0.9,0.85,4,279,6,0,0,support,medium,1.0,2.0
7,0,0.92,0.77,5,191,2,0,0,technical,medium,1.0,1.0
8,0,0.62,0.78,3,178,3,1,0,technical,low,0.0,1.0
9,0,0.83,0.89,4,136,3,0,0,sales,medium,1.0,0.0


In [ ]:
# Création d'une Pipeline
indexer = Pipeline(stages =  [SalaryIndexer, SalesIndexer])
# Indexer les variables de hr
hrIndexed = indexer.fit(hr).transform(hr)
# Affichage d'un extrait de hrIndexed
hrIndexed.sample(False, 0.001 , seed = 222).toPandas()

In [23]:
hrIndexed.columns


['left',
 'satisfaction_level',
 'last_evaluation',
 'number_project',
 'average_montly_hours',
 'time_spend_company',
 'Work_accident',
 'promotion_last_5years',
 'sales',
 'salary',
 'indexedSalary',
 'indexedSales']

In [25]:
# (a) Créer une base de données hrNumeric excluant les anciennes variables non indexées.
hrNumeric= hrIndexed.select(['left','satisfaction_level','last_evaluation', 'number_project','average_montly_hours','time_spend_company','Work_accident',
 'promotion_last_5years',
 'indexedSalary',
 'indexedSales'])
# (c) Afficher un extrait de la nouvelle base de données.

In [26]:
# Import de DenseVector du package pyspark.ml.linalg
from pyspark.ml.linalg import DenseVector

In [27]:
# (b) Créer hrLibsvm, une base de données au format svmlib à partir de hrNumeric.
# Création d'une variable DenseVector contenant les features en passant par la structure RDD
hrRdd = hrNumeric.rdd.map(lambda x: (x[0], DenseVector(x[1:])))

In [28]:
# Transformation en DataFrame et nommage des variables pour obtenir une base au format svmlib
hrLibsvm = spark.createDataFrame(hrRdd, ['label', 'features'])

In [ ]:
# Affichage d'un extrait de hrLibsvm
hrLibsvm.sample(False, .001, seed = 222).toPandas()

#vérifier le nombre de modalités pour la variable label


In [30]:
hrNumeric.describe().toPandas()

,summary,left,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,indexedSalary,indexedSales
0,count,14999,14999,14999,14999,14999,14999,14999,14999,14999,14999
1,mean,0.2380825388359224,0.6128335222348166,0.7161017401159978,3.80305353690246,201.0503366891126,3.498233215547703,0.1446096406427095,0.021268084538969265,0.5947063137542503,2.69551303420228
2,stddev,0.42592409938029885,0.24863065106114257,0.17116911062327556,1.2325923553183513,49.94309937128406,1.4601362305354808,0.35171855238017957,0.1442814645785825,0.6371829504695818,2.754845263313967
3,min,0,0.09,0.36,2,100,10,0,0,0.0,0.0
4,max,1,1,1,7,99,8,1,1,2.0,9.0


In [32]:
# Nous avons 2 variables catégorielles: "indexedSales" et "indexedSalary"
# On peut obtenir le nombre de modalités en regardant 'min' et 'max' des variables indexées:
# 'sales' possède le plus de modalités et en possède 10 (10 entiers entre 0.0 et 9.0)
# Import de VectorIndexer du package pyspark.ml.feature
from pyspark.ml.feature import VectorIndexer
# Création d'un transformateur indexant les features
featureIndexer = VectorIndexer(inputCol="features",
                               outputCol="indexedFeatures",
                               maxCategories = 10).fit(hrLibsvm)

In [33]:
# Import du classifieur RandomForestClassifier du package pyspark.ml.classification
from pyspark.ml.classification import RandomForestClassifier
# Création des transformateurs
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(hrLibsvm)
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories = 10).fit(hrLibsvm)

# Création d'un classifieur
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", predictionCol='prediction', seed = 222)

# Création d'un transformateur permettant de rétablir les labels des prédictions
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)
# Création d'une Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf, labelConverter])

# Décomposition des données en deux ensembles: données d'entraînement et de test
(train, test) = hrLibsvm.randomSplit([0.7, 0.3], seed = 222)

# Apprentissage du modèle en utilisant les données d'entraînement
model = pipeline.fit(train)

In [34]:
# Calcul des prédictions
predictions = model.transform(test)
# Affichage d'un extrait des prédictions
predictions.sample(False, 0.001 , seed = 222).toPandas()

,label,features,indexedLabel,indexedFeatures,rawPrediction,probability,prediction,predictedLabel
0,0,"[0.77, 0.64, 3.0, 190.0, 10.0, 1.0, 0.0, 1.0, ...",0.0,"[0.77, 0.64, 1.0, 190.0, 7.0, 1.0, 0.0, 1.0, 3.0]","[19.751862785755634, 0.24813721424436633]","[0.9875931392877817, 0.012406860712218316]",0.0,0
1,0,"[0.92, 0.88, 4.0, 111.0, 4.0, 0.0, 0.0, 1.0, 1.0]",0.0,"[0.92, 0.88, 2.0, 111.0, 2.0, 0.0, 0.0, 1.0, 1.0]","[18.12047351777507, 1.8795264822249274]","[0.9060236758887535, 0.09397632411124637]",0.0,0


In [35]:

# 5. Evaluation du modèle
# Une fois le modèle d'apprentissage automatique construit, il est important de vérifier la fiabilité des prédictions, à la fois pour le comparer à d'autres modèles de classification mais également pour optimiser ses paramètres.

# Pour cela, il existe un sous-module pyspark.ml.evaluation contenant toutes les métriques d'évaluation. En particulier, vous y trouverez la fonction MulticlassClassificationEvaluator permettant d'évaluer des modèles de classification.

# Cette fonction prend 3 arguments principaux :

# metricName : métrique à utiliser, typiquement : 'accuracy'.
# labelCol : nom de la colonne à prédire.
# predictionCol : nom de la colonne de prédictions.
# L'évaluateur créé possède une méthode evaluate permettant de l'appliquer à un échantillon.

# (a) Importer la fonction MulticlassClassificationEvaluator.
# (b) Créer evaluator l'évaluateur d'accuracy du modèle.
# (c) Calculer et afficher la précision accuracy de la prédiction sur l'échantillon test.
#
# Import d'un évaluateur MulticlassClassificationEvaluator du package pyspark.ml.evaluation
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Création d'un évaluateur
evaluator = MulticlassClassificationEvaluator(metricName='accuracy',
                                              labelCol= 'indexedLabel',
                                              predictionCol= 'prediction')
# Calcul et affichage de la précision du modèle
accuracy = evaluator.evaluate(predictions)
print(accuracy)

0.9639821029082773
